<h1><center>Discussion of potential starmap extensions</center></h1>

This notebook discusses extensions to the starmap code which would be required for our project.
1. 3D implementation of the Starmap staggered grid solver
2. Anisotropic radiative transfer equation (ARTE)
3. Steady state solution


----------------

### 3D implementation of the Starmap staggered grid solver

A 3D version of starmap actually exists, but is not open source. However, extending starmap for 3D problems appears rather straight-forward. The 2D version already solves the general 3D equations. The difference is, that in the 2D version, all moment coefficients for which $l+m$ are odd vanish in the solution vector. Therefore respective rows are left out in the matrices $M_x$, $M_y$ and $S$ and also in the moment coefficients for the phase function. Extending the 2D starmap code to 3D requires the following steps:

1. Do not drop the coefficients for which $l+m$ are odd from the solution vector and from the matrices $M_x$, $M_y$ and $S$. These matrices now will also model the dependencies on x- and y-derivatives of these new coefficients.
2. Introduce the complex-valued matrix $M_z$ (and its real variable pendant $SM_zS^{-1}$) by considering the $\partial_z$ term in the derivation of the SH moment equations. This new matrix defines the dependecies of moment coefficients on z-derivatives from other coefficients.
3. Introduce a new staggered grid location at the z-edge of the voxel and adopt the staggered grid generation code accordingly.
4. The zero moments for $\sigma_a$ and $\sigma_s$ now need to be computed for the new grid locations as well.
5. During time stepping, we now need to compute the z-derivatives of all solution grids (which live at different staggered grid locations) and also the x- and y- derivatives of the grids at the newly introduced staggered grid location.
6. We need to substract $M_z\partial_z\vec{u}_k$ when computing $\vec{u}_{k+1}$.


----------------

### Anisotropic radiative transfer equation

The anisotropic radiative transfer equation is given as:

$$
\begin{align}
(\omega\cdot\nabla)L(\vec{x}, \omega)=-\sigma_t(\vec{x}, \omega)L(\vec{x}, \omega) + \sigma_s(\vec{x}, \omega)\int_{S^2}f_p(\vec{x}, \omega'\rightarrow\omega)L(\vec{x}, \omega')\mathbf{d}\omega' + Q(\vec{x}, \omega)
\end{align}
$$

We now will derive the spherical harmonics expansion of each individual term.

##### The directional derivative term

The directional derivative term $(\omega\cdot\nabla)L(\vec{x}, \omega)$ is the same as in the isotropic RTE. Therefore its SH expansion will also be the same. We refer to the notebook spherical_harmonics_method.ipynb for the derivation.

The SH expansion of the directional derivative term gives rise to the matrices $M_x$, $M_y$ and $M_z$, which model the dependencies of SH coefficients on spatial derivatives from other SH coefficients. These matrices will be the same for the ARTE and can be expected to have the same structure which allows for the staggered grid approach.

##### The extinction term

The extinction term is given as:

$$
-\sigma_t(\vec{x}, \omega)L(\vec{x}, \omega)
$$

We first replace both, radiance field and extinction coefficient with its SH expansions:
$$
-\left(\sum_{l_1}\sum_{m_1=-l_1}^{l_1}\sigma_t^{l_1m_1}\left(\vec{x}\right)Y^{l_1m_1}\left(\omega\right)\right)\left(\sum_{l_2}\sum_{m_2=-l_2}^{l_2} L^{l_2m_2}\left(\vec{x}\right)Y^{l_2m_2}\left(\omega\right)\right)
$$

After rearranging we get:

$$
-\sum_{l_1}\sum_{m_1=-l_1}^{l_1}\sum_{l_2}\sum_{m_2=-l_2}^{l_2}\sigma_t^{l_1m_1}\left(\vec{x}\right)L^{l_2m_2}\left(\vec{x}\right)Y^{l_1m_1}\left(\omega\right)Y^{l_2m_2}\left(\omega\right)
$$

It appears, that there is a way of expressing the product of the spherical harmonics expansions of two functions in terms of spherical harmonics. This relationship is given by the [Clebsch-Gordan coefficients](https://en.wikipedia.org/wiki/Clebsch%E2%80%93Gordan_coefficients#Relation_to_spherical_harmonics):

$$
Y^{l_1m_1}\left(\omega\right)Y^{l_2m_2}\left(\omega\right) = \sum_{l}\sum_{m=-l}^{l}\sqrt{\frac{\left(2l_1+1\right)\left(2l_2+1\right)}{4\pi\left(2l+1\right)}}C_{l_10l_20}^{l0}C_{l_1m_1l_2m_2}^{lm}Y^{lm}\left(\omega\right)
$$

Applying this to our expanded coefficient term gives:

$$
-\sum_{l_1}\sum_{m_1=-l_1}^{l_1}\sum_{l_2}\sum_{m_2=-l_2}^{l_2}\sum_{l}\sum_{m=-l}^{l}\sigma_t^{l_1m_1}\left(\vec{x}\right)L^{l_2m_2}
\sqrt{\frac{\left(2l_1+1\right)\left(2l_2+1\right)}{4\pi\left(2l+1\right)}}C_{l_10l_20}^{l0}C_{l_1m_1l_2m_2}^{lm}Y^{lm}\left(\omega\right)
$$

Now we have expressed the quantities in the extinction term in SH but it still depends on direction $\omega$. Applying the second step of multiplying with $\overline{Y^{l'm'}}$ and integrating over solid angle gives:

$$
-\sum_{l_1}\sum_{m_1=-l_1}^{l_1}\sum_{l_2}\sum_{m_2=-l_2}^{l_2}\sum_{l}\sum_{m=-l}^{l}\sigma_t^{l_1m_1}\left(\vec{x}\right)L^{l_2m_2}
\sqrt{\frac{\left(2l_1+1\right)\left(2l_2+1\right)}{4\pi\left(2l+1\right)}}C_{l_10l_20}^{l0}C_{l_1m_1l_2m_2}^{lm}\int_{\Omega}\overline{Y^{l'm'}}\left(\omega\right)Y^{lm}\left(\omega\right)\mathbf{d}\omega
$$

Applying the orthogonality property gives:

$$
-\sum_{l_1}\sum_{m_1=-l_1}^{l_1}\sum_{l_2}\sum_{m_2=-l_2}^{l_2}
\sigma_t^{l_1m_1}\left(\vec{x}\right)L^{l_2m_2}
\sqrt{\frac{\left(2l_1+1\right)\left(2l_2+1\right)}{4\pi\left(2l+1\right)}}C_{l_10l_20}^{l0}C_{l_1m_1l_2m_2}^{lm}
$$

We swap the SH indices (and change the order of summation sigmas), such that we have an expression in terms of $L^{lm}$:

$$
-\sum_{l}\sum_{m=-l}^{l}
\left(
\sum_{l_1}\sum_{m_1=-l_1}^{l_1}
\sigma_t^{l_1m_1}\left(\vec{x}\right)
\sqrt{\frac{\left(2l_1+1\right)\left(2l+1\right)}{4\pi\left(2l_2+1\right)}}C_{l_1 0 l 0}^{l_2 0}C_{l_1m_1lm}^{l_2m_2}
\right)
L^{lm}
$$

It seems that this expression is a matrix product with the solution vector (complex-variables) and a diagonal matrix, where the diagonal matrix elements are computed (once) from the different moment coefficients of extinction and Clebsch-Gordan coefficients. This matrix can be still be turned into real variables using $S$.

However, there is one important thing to consider: We have seen that the Clebsch-Gordan expression for the product between SH basis functions has _higher order_ than the original basis functions, which went into the product. In general, the order of $Y^{l_1m_1}Y^{l_2m_2}$ is bound by $l_1+l_2$.

In other words: our simulation will not account for higher-(than-our-solver-)order frequencies, which are produced by the interaction between anisotropic extinction coefficient and the radiance field.




##### The scattering term (simplified)

In this section, we do the SH-expansion of the scattering term of the ARTE. Without loss of generality, we simplify the term by folding the anisotropic scattering coefficient $\sigma_s(\vec{x}, \omega)$ into the phase function $f_p$. The scattering term is then given as:

$$
\int_{S^2}f_p(\vec{x}, \omega'\rightarrow\omega)L(\vec{x}, \omega')\mathbf{d}\omega'
$$

Here we face the problem of an integral over the product of the fully anisotropic phase function $f_p$ and the directional-dependent radiance field $L$.

Following the general procedure for the moment expansion, we would first like to replace each quantity ($f_p$ and $L$) with its spherical harmonic expansion. In the starmap paper from Seibold, the fact that the phase function was rotationally invariant was crucial, because it allowed to express the integral as a convolution of the radiance field with the rotationally symmetric phase function. This convolution in turn could be expressed as a product between SH coefficients (see scattering term derivation in spherical_harmonics_method.ipynb).

Now in our case the phase function is not rotationally invariant anymore. Therefore, the integral in the scattering term does not easily collapse to a product between SH coefficients. To get a handle on the problem, we first need to represent the phase function in terms of SH basis functions.

We follow the strategy in [this paper](http://dl.acm.org/citation.cfm?id=134075), which derives the general SH representation for anisotropic BRDF's. A problem very similar to ours, with the only difference that our domain is the full sphere, not only the hemisphere, and our phase function does not satisfy reciprocity.

Let us write the phase function $f_p( \omega_i,\omega_o )$ in terms of incident direction $\omega_i$ and outgoing direction $\omega_o$. Now if we hold $\omega_i$ fixed, the phase function reduces to a spherical function, which can be easily represented in SH:

$$
f_p(\omega_i, \omega_o) = \sum_l\sum_{m=-l}^l f_p^{lm}\left(\omega_i\right) Y^{lm}\left(\omega_o\right)
$$

Now the key idea is to realize that the coefficient $f_p^{lm}$ is another spherical function and therefore can also be easily represented in SH:

$$
f_p(\omega_i, \omega_o) = \sum_l\sum_{m=-l}^l \sum_{l'}\sum_{m'=-l'}^{l'} f_p^{lml'm'} Y^{lm}\left(\omega_i\right) Y^{l'm'}\left(\omega_o\right)
$$

Thus, anisotropic phase functions are represented by a square matrix of coefficients, where the size of the matrix is determined by the SH truncation order. See sggx_sh.ipynb for an implementation and further studies.

We now can replace the quantities in the scattering term by their SH expansions:

$$
\int_{S^2} \left(\sum_{l_1}\sum_{m_1=-l_1}^{l_1} \sum_{l_2}\sum_{m_2=-l_2}^{l_2} f_p^{l_1m_1l_2m_2}\left(\vec{x}\right) Y^{l_1m_1}\left(\omega_i\right) Y^{l_2m_2}\left(\omega_o\right)\right)\left( \sum_{l_3}\sum_{m_3=-l_3}^{l_3} L^{l_3m_3}(\vec{x})Y^{l_3m_3}\left(\omega_i\right)\right) \mathbf{d}\omega_i
$$

After some rearrangement we get:

$$
\sum_{l_1}\sum_{m_1=-l_1}^{l_1} \sum_{l_2}\sum_{m_2=-l_2}^{l_2}\sum_{l_3}\sum_{m_3=-l_3}^{l_3}f_p^{l_1m_1l_2m_2}\left(\vec{x}\right) L^{l_3m_3}(\vec{x})Y^{l_2m_2}\left(\omega_o\right)\int_{S^2} Y^{l_1m_1}\left(\omega_i\right) Y^{l_3m_3}\left(\omega_i\right) \mathbf{d}\omega_i
$$


We apply the following identity (see spherical_harmonics_properties.ipynb for validation):

$$
\begin{align}
\int_\Omega Y^{l_1m_1}\left(\omega\right)Y^{l_2 m_2}\left(\omega\right)\mathbf{d}\omega&=\left(-1\right)^{m_1}\delta_{l_1l_2}\delta_{(m_1+m_2)0}
\end{align}
$$


This gives

$$
\sum_{l_1}\sum_{m_1=-l_1}^{l_1}
\sum_{l_2}\sum_{m_2=-l_2}^{l_2}
\sum_{l_3}\sum_{m_3=-l_3}^{l_3}
f_p^{l_1m_1l_2m_2}\left(\vec{x}\right) L^{l_3m_3}(\vec{x})Y^{l_2 m_2}\left(\omega_o\right)
\left(-1\right)^{m_1}\delta_{l_1l_3}\delta_{(m_1+m_3)0}
$$

We now have expressed the interaction between quantities in the scattering term with spherical harmonics. However, the term still depends on direction. We follow the procedure of multiplying with $\overline{Y^{lm}}$ and integrating over solid angle $\omega_o$. Applying the identity $\int_\Omega\overline{Y^{lm}}(\omega)Y^{l_2m_2}\left(\omega\right)\mathbf{d}\omega = \delta_{ll_2}\delta_{mm_2}$ gives:

$$
\sum_{l_1}\sum_{m_1=-l_1}^{l_1}
\sum_{l_3}\sum_{m_3=-l_3}^{l_3}
f_p^{l_1m_1lm}\left(\vec{x}\right) L^{l_3m_3}(\vec{x})
\left(-1\right)^{m_1}\delta_{l_1l_3}\delta_{(m_1+m_3)0}
$$

we rearrange terms into a dot product form:

$$
\sum_{l_3}\sum_{m_3=-l_3}^{l_3}
\left(
\sum_{l_1}\sum_{m_1=-l_1}^{l_1}
f_p^{l_1m_1lm}\left(\vec{x}\right)
\left(-1\right)^{m_1}\delta_{l_1l_3}\delta_{(m_1+m_3)0}
\right)
L^{l_3m_3}(\vec{x})
$$

Accounting for the $\delta_{l_1l_3}$-term will further reduce the expression to:

$$
\sum_{l_3}\sum_{m_3=-l_3}^{l_3}
\left(
\sum_{m_1=-l_1}^{l_3}
f_p^{l_3m_1lm}\left(\vec{x}\right)
\left(-1\right)^{m_1}\delta_{(m_1+m_3)0}
\right)
L^{l_3m_3}(\vec{x})
$$

The equation associated with coefficient $l$, $m$, consists of a linear combination of coefficients. Therefore we can write down a matrix $M_s$ for our system of linear equations, which is similar to $M_x$ or $M_y$. It couples the coefficients between each other. This coupling is driven by the phase function. The scattering term becomes:

$$
M_s\vec{L}
$$

where $\vec{L}$ is the solution vector holding the coefficients $L^{lm}$. For an isotropic phase function, $M_s$ will only have its $0,0$-element to be a non-zero element. For a rotationally invariant phase function, $M_s$ will be a diagonal matrix. For general rotationally variant phase functions, the structure of $M_s$ will be more dense. See starmap_extensions_arte_study.ipynb for a more detailed analysis.


In the starmap paper, $M_s$ (which was folded into $C$) was diagonal. This is not the case with general anisotropic phase functions and poses the following challenges on the method:

1. We need to convert $M_s$ to real variables at every gridpoint. This is done by multiplying with $S$ and should be no big deal.
2. For general phase functions $M_s$ couples coefficients in a way, such that even coefficients depend on even coefficients and odd coefficients depend odd coefficients. This violates a core assumption of the staggered grid approach, where the update of even components is done by only considering odd components and vice versa. 
3. The original solution method uses matrix exponentials of $M_s$ during time stepping. Since $M_s$ is diagonal, the matrix exponential is simply found by taking the exponential of each entry of the main diagonal (see [here](https://en.wikipedia.org/wiki/Matrix_exponential#Diagonalizable_case)). In our case, $M_s$ is not diagonal. Computing the exponential then becomes more difficult and is often done using some approximation technique (see [here](https://en.wikipedia.org/wiki/Matrix_exponential#Computing_the_matrix_exponential)). However, even if this is more involved, it could still be done in a pre-processing step for non-time dependent problems.

----------------

### Steady state solution from staggered grid solver

In his email, Seibold mentioned that they made some progress for doing implicit solves of individual time steps (which would also allow steady state solves). However, I wasn't able to find the slides he mentioned (I have sent him an email about this
).

Without any more information I currently have no idea where to start here. I probably should read into exponential integrators (I assume this is what they use in starmap) and how to turn them implicit.